In [1]:
from GUI import GUI
import layout.lib.repetition_recognition as rep

In [56]:
gui = GUI(img_file='data/input/v2.jpg')

In [57]:
gui.detect_element()
# gui.load_detection_result()
gui.visualize_element_detection()

[Text Detection Completed in 0.974 s] Input: data/input/v2.jpg Output: data/output\ocr\v2.json
[Compo Detection Completed in 0.594 s] Input: data/input/v2.jpg Output: data/output\ip\v2.json
[Merge Completed] Input: data/input/v2.jpg Output: data/output\uied\v2.jpg


In [58]:
gui.recognize_layout()
gui.visualize_layout_recognition()

[Layout Recognition Completed in 1.425 s] Input: data/input/v2.jpg Output: data/output\layout\v2.json


In [85]:
gui.visualize_element_detection()

In [70]:
gui.visualize_layout_recognition()